# Volumeter
To start up the system, run the cell below with **Shift** + **Enter**.

In [1]:
import import_ipynb
from backend.Backend_notebook import *
start()

Selected holder: Vanha_häkki_2020-01-14T22:21:14.630742


# TODO
- Kommenttikenttä
- Hyväksy mittaus
- Laske tilavuus
- Leikkaa diff jos jompi kumpi data loppuu kesken